In [ ]:
JENZA_COURSES_HTML_FILE_PATH = "./.localdata/raw__jenzabar.html"

In [47]:
import bs4
with open(JENZA_COURSES_HTML_FILE_PATH) as f:
  raw_html = f.read()

In [48]:
soup = bs4.BeautifulSoup(raw_html, "html.parser")
courses_table = soup.find("table")
col_names_row = courses_table.find_all("tr")[0]
col_names = [c.text.strip() for c in col_names_row.find_all("th")]
col_names

['Course',
 'Section',
 'Session',
 'Credits',
 'Campus',
 'Instructor',
 'Times',
 'Taken/Seats',
 'Spaces Waiting',
 'Delivery Method',
 'Dist. Learning',
 'Location']

In [49]:
course_data_rows = courses_table.find_all("tr")[1:]
courses = []
for r in course_data_rows:
  course_data = [c.text.strip() for c in r.find_all("td")]
  courses.append(course_data)

courses[0]

['Bridge Course II - Nursing Skills II(BSN020)',
 '0',
 '15w',
 '0',
 'AUA Main Campus',
 'Vicky Herkelian',
 'TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm',
 '0/20',
 '0',
 '',
 '',
 '201N Najarian, 205N Sim. Lab Najarian']

In [50]:
import pandas as pd

courses_df = pd.DataFrame(courses, columns=col_names)
del courses_df["Dist. Learning"]
del courses_df["Delivery Method"]
del courses_df["Spaces Waiting"]
del courses_df["Taken/Seats"]
courses_df

,Course,Section,Session,Credits,Campus,Instructor,Times,Location
0,Bridge Course II - Nursing Skills II(BSN020),0,15w,0,AUA Main Campus,Vicky Herkelian,TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm,"201N Najarian, 205N Sim. Lab Najarian"
1,Professional Nursing Practice(BSN101),UNQ,15w,3,AUA Main Campus,Nour Alayan,"MON 3:30pm-4:20pm, WED 3:30pm-5:20pm",Classroom 407E Paramaz Avedissian Building
2,Patophysiology(BSN200),UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"TUE 5:00pm-6:15pm, THU 5:00pm-6:15pm",Classroom 407E Paramaz Avedissian Building
3,Health Assessment Across Age Groups(BSN201),UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"MON 4:30pm-6:20pm, WED 6:00pm-6:50pm WED 6:00p...","201N Najarian, 205N Sim. Lab Najarian, Classro..."
4,Research in Nursing(BSN205),UNQ,15w,3,AUA Main Campus,Adena Alahverdian,"TUE 4:30pm-5:45pm, THU 4:30pm-5:45pm",Classroom 406E Paramaz Avedissian Building
...,...,...,...,...,...,...,...,...
530,English for Specific Purposes(TEFL308)(Prerequ...,0,15w,3,AUA Main Campus,Irshat Madyarov,WED 9:30am-12:20pm,202N Najarian
531,Teaching Internship(TEFL310),0,15w,3,AUA Main Campus,Talin Grigorian,TBD,
532,Teaching Test Preparation(TEFL312),0,15w,2,AUA Main Campus,Irena Galikyan,THU 3:00pm-4:15pm,Classroom 216M Main Building
533,Introduction to Language(TEFL320),0,15w,2,AUA Main Campus,Rubina Gasparyan,WED 12:30pm-2:20pm,202N Najarian


In [51]:
def get_first_number_index(full_subject_code: str):
  for i in range(len(full_subject_code)):
    if full_subject_code[i].isnumeric():
      return i
  return -1

for i in courses_df.index:
  course_data = courses_df.loc[i]
  course_name = course_data['Course']
  full_subject_code = course_name.split("(")[1].strip()[:-1]
  courses_df.loc[i, 'Subject Code'] = full_subject_code[:get_first_number_index(full_subject_code)]
  courses_df.loc[i, 'Course Number'] = full_subject_code[get_first_number_index(full_subject_code):]
  courses_df.loc[i, 'Course Title'] = course_name.split("(")[0].strip().replace("\n", "")

courses_df[:3]

,Course,Section,Session,Credits,Campus,Instructor,Times,Location,Subject Code,Course Number,Course Title
0,Bridge Course II - Nursing Skills II(BSN020),0,15w,0,AUA Main Campus,Vicky Herkelian,TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm,"201N Najarian, 205N Sim. Lab Najarian",BSN,020,Bridge Course II - Nursing Skills II
1,Professional Nursing Practice(BSN101),UNQ,15w,3,AUA Main Campus,Nour Alayan,"MON 3:30pm-4:20pm, WED 3:30pm-5:20pm",Classroom 407E Paramaz Avedissian Building,BSN,101,Professional Nursing Practice
2,Patophysiology(BSN200),UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"TUE 5:00pm-6:15pm, THU 5:00pm-6:15pm",Classroom 407E Paramaz Avedissian Building,BSN,200,Patophysiology


In [52]:
del courses_df["Course"]
courses_df

,Section,Session,Credits,Campus,Instructor,Times,Location,Subject Code,Course Number,Course Title
0,0,15w,0,AUA Main Campus,Vicky Herkelian,TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm,"201N Najarian, 205N Sim. Lab Najarian",BSN,020,Bridge Course II - Nursing Skills II
1,UNQ,15w,3,AUA Main Campus,Nour Alayan,"MON 3:30pm-4:20pm, WED 3:30pm-5:20pm",Classroom 407E Paramaz Avedissian Building,BSN,101,Professional Nursing Practice
2,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"TUE 5:00pm-6:15pm, THU 5:00pm-6:15pm",Classroom 407E Paramaz Avedissian Building,BSN,200,Patophysiology
3,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"MON 4:30pm-6:20pm, WED 6:00pm-6:50pm WED 6:00p...","201N Najarian, 205N Sim. Lab Najarian, Classro...",BSN,201,Health Assessment Across Age Groups
4,UNQ,15w,3,AUA Main Campus,Adena Alahverdian,"TUE 4:30pm-5:45pm, THU 4:30pm-5:45pm",Classroom 406E Paramaz Avedissian Building,BSN,205,Research in Nursing
...,...,...,...,...,...,...,...,...,...,...
530,0,15w,3,AUA Main Campus,Irshat Madyarov,WED 9:30am-12:20pm,202N Najarian,TEFL,308,English for Specific Purposes
531,0,15w,3,AUA Main Campus,Talin Grigorian,TBD,,TEFL,310,Teaching Internship
532,0,15w,2,AUA Main Campus,Irena Galikyan,THU 3:00pm-4:15pm,Classroom 216M Main Building,TEFL,312,Teaching Test Preparation
533,0,15w,2,AUA Main Campus,Rubina Gasparyan,WED 12:30pm-2:20pm,202N Najarian,TEFL,320,Introduction to Language


In [53]:
for i in courses_df.index:
    # cleanup the course titles
    courses_df.at[i, 'Course Title'] = courses_df.at[i, 'Course Title'].strip()

courses_df

,Section,Session,Credits,Campus,Instructor,Times,Location,Subject Code,Course Number,Course Title
0,0,15w,0,AUA Main Campus,Vicky Herkelian,TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm,"201N Najarian, 205N Sim. Lab Najarian",BSN,020,Bridge Course II - Nursing Skills II
1,UNQ,15w,3,AUA Main Campus,Nour Alayan,"MON 3:30pm-4:20pm, WED 3:30pm-5:20pm",Classroom 407E Paramaz Avedissian Building,BSN,101,Professional Nursing Practice
2,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"TUE 5:00pm-6:15pm, THU 5:00pm-6:15pm",Classroom 407E Paramaz Avedissian Building,BSN,200,Patophysiology
3,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"MON 4:30pm-6:20pm, WED 6:00pm-6:50pm WED 6:00p...","201N Najarian, 205N Sim. Lab Najarian, Classro...",BSN,201,Health Assessment Across Age Groups
4,UNQ,15w,3,AUA Main Campus,Adena Alahverdian,"TUE 4:30pm-5:45pm, THU 4:30pm-5:45pm",Classroom 406E Paramaz Avedissian Building,BSN,205,Research in Nursing
...,...,...,...,...,...,...,...,...,...,...
530,0,15w,3,AUA Main Campus,Irshat Madyarov,WED 9:30am-12:20pm,202N Najarian,TEFL,308,English for Specific Purposes
531,0,15w,3,AUA Main Campus,Talin Grigorian,TBD,,TEFL,310,Teaching Internship
532,0,15w,2,AUA Main Campus,Irena Galikyan,THU 3:00pm-4:15pm,Classroom 216M Main Building,TEFL,312,Teaching Test Preparation
533,0,15w,2,AUA Main Campus,Rubina Gasparyan,WED 12:30pm-2:20pm,202N Najarian,TEFL,320,Introduction to Language


In [54]:
for i in courses_df.index:
    is_najarian = "najar" in courses_df.loc[i, 'Location'].lower()
    is_main_building = "main" in courses_df.loc[i, 'Location'].lower()
    is_pab = "paramaz" in courses_df.loc[i, 'Location'].lower() or "PAB" in courses_df.loc[i, 'Location'].lower()

    if is_najarian:
        courses_df.at[i, 'Building'] = "AUA Najarian Center"
    elif is_main_building:
        courses_df.at[i, 'Building'] = "Main Building"
    elif is_pab:
        courses_df.at[i, 'Building'] = "Paramaz Avedisian Building"
    else:
        courses_df.at[i, 'Building'] = "Other"
courses_df

,Section,Session,Credits,Campus,Instructor,Times,Location,Subject Code,Course Number,Course Title,Building
0,0,15w,0,AUA Main Campus,Vicky Herkelian,TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm,"201N Najarian, 205N Sim. Lab Najarian",BSN,020,Bridge Course II - Nursing Skills II,AUA Najarian Center
1,UNQ,15w,3,AUA Main Campus,Nour Alayan,"MON 3:30pm-4:20pm, WED 3:30pm-5:20pm",Classroom 407E Paramaz Avedissian Building,BSN,101,Professional Nursing Practice,Paramaz Avedisian Building
2,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"TUE 5:00pm-6:15pm, THU 5:00pm-6:15pm",Classroom 407E Paramaz Avedissian Building,BSN,200,Patophysiology,Paramaz Avedisian Building
3,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"MON 4:30pm-6:20pm, WED 6:00pm-6:50pm WED 6:00p...","201N Najarian, 205N Sim. Lab Najarian, Classro...",BSN,201,Health Assessment Across Age Groups,AUA Najarian Center
4,UNQ,15w,3,AUA Main Campus,Adena Alahverdian,"TUE 4:30pm-5:45pm, THU 4:30pm-5:45pm",Classroom 406E Paramaz Avedissian Building,BSN,205,Research in Nursing,Paramaz Avedisian Building
...,...,...,...,...,...,...,...,...,...,...,...
530,0,15w,3,AUA Main Campus,Irshat Madyarov,WED 9:30am-12:20pm,202N Najarian,TEFL,308,English for Specific Purposes,AUA Najarian Center
531,0,15w,3,AUA Main Campus,Talin Grigorian,TBD,,TEFL,310,Teaching Internship,Other
532,0,15w,2,AUA Main Campus,Irena Galikyan,THU 3:00pm-4:15pm,Classroom 216M Main Building,TEFL,312,Teaching Test Preparation,Main Building
533,0,15w,2,AUA Main Campus,Rubina Gasparyan,WED 12:30pm-2:20pm,202N Najarian,TEFL,320,Introduction to Language,AUA Najarian Center


In [58]:
import dotenv
from openai import OpenAI

dotenv.load_dotenv()
client = OpenAI()

for i in courses_df.index:
    location = courses_df.at[i, 'Location']
    if location:
        response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Extract the room(s) from the given location: {location}. Only return the room(s) without any additional text."}
            ]
        )
        room = response.choices[0].message.content.strip()
        courses_df.at[i, 'Room'] = room

courses_df

,Section,Session,Credits,Campus,Instructor,Times,Location,Subject Code,Course Number,Course Title,Building,Room
0,0,15w,0,AUA Main Campus,Vicky Herkelian,TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm,"201N Najarian, 205N Sim. Lab Najarian",BSN,020,Bridge Course II - Nursing Skills II,AUA Najarian Center,201N\n205N
1,UNQ,15w,3,AUA Main Campus,Nour Alayan,"MON 3:30pm-4:20pm, WED 3:30pm-5:20pm",Classroom 407E Paramaz Avedissian Building,BSN,101,Professional Nursing Practice,Paramaz Avedisian Building,Room 407E
2,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"TUE 5:00pm-6:15pm, THU 5:00pm-6:15pm",Classroom 407E Paramaz Avedissian Building,BSN,200,Patophysiology,Paramaz Avedisian Building,407E
3,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"MON 4:30pm-6:20pm, WED 6:00pm-6:50pm WED 6:00p...","201N Najarian, 205N Sim. Lab Najarian, Classro...",BSN,201,Health Assessment Across Age Groups,AUA Najarian Center,201N\n205N\n407E
4,UNQ,15w,3,AUA Main Campus,Adena Alahverdian,"TUE 4:30pm-5:45pm, THU 4:30pm-5:45pm",Classroom 406E Paramaz Avedissian Building,BSN,205,Research in Nursing,Paramaz Avedisian Building,406E
...,...,...,...,...,...,...,...,...,...,...,...,...
530,0,15w,3,AUA Main Campus,Irshat Madyarov,WED 9:30am-12:20pm,202N Najarian,TEFL,308,English for Specific Purposes,AUA Najarian Center,202N
531,0,15w,3,AUA Main Campus,Talin Grigorian,TBD,,TEFL,310,Teaching Internship,Other,NaN
532,0,15w,2,AUA Main Campus,Irena Galikyan,THU 3:00pm-4:15pm,Classroom 216M Main Building,TEFL,312,Teaching Test Preparation,Main Building,216M
533,0,15w,2,AUA Main Campus,Rubina Gasparyan,WED 12:30pm-2:20pm,202N Najarian,TEFL,320,Introduction to Language,AUA Najarian Center,202N


In [ ]:
del courses_df["Location"]

In [ ]:
courses_df["Room"] = courses_df["Room"].str.strip().replace("\n", ",").replace(" ", ",").replace("\\n", ",")

In [67]:
courses_df

,Section,Session,Credits,Campus,Instructor,Times,Subject Code,Course Number,Course Title,Building,Room
0,0,15w,0,AUA Main Campus,Vicky Herkelian,TUE 5:00pm-8:00pm TUE 5:00pm-8:00pm,BSN,020,Bridge Course II - Nursing Skills II,AUA Najarian Center,201N\n205N
1,UNQ,15w,3,AUA Main Campus,Nour Alayan,"MON 3:30pm-4:20pm, WED 3:30pm-5:20pm",BSN,101,Professional Nursing Practice,Paramaz Avedisian Building,Room 407E
2,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"TUE 5:00pm-6:15pm, THU 5:00pm-6:15pm",BSN,200,Patophysiology,Paramaz Avedisian Building,407E
3,UNQ,15w,3,AUA Main Campus,Mohamed Shehnaaz,"MON 4:30pm-6:20pm, WED 6:00pm-6:50pm WED 6:00p...",BSN,201,Health Assessment Across Age Groups,AUA Najarian Center,201N\n205N\n407E
4,UNQ,15w,3,AUA Main Campus,Adena Alahverdian,"TUE 4:30pm-5:45pm, THU 4:30pm-5:45pm",BSN,205,Research in Nursing,Paramaz Avedisian Building,406E
...,...,...,...,...,...,...,...,...,...,...,...
530,0,15w,3,AUA Main Campus,Irshat Madyarov,WED 9:30am-12:20pm,TEFL,308,English for Specific Purposes,AUA Najarian Center,202N
531,0,15w,3,AUA Main Campus,Talin Grigorian,TBD,TEFL,310,Teaching Internship,Other,NaN
532,0,15w,2,AUA Main Campus,Irena Galikyan,THU 3:00pm-4:15pm,TEFL,312,Teaching Test Preparation,Main Building,216M
533,0,15w,2,AUA Main Campus,Rubina Gasparyan,WED 12:30pm-2:20pm,TEFL,320,Introduction to Language,AUA Najarian Center,202N


In [ ]:
courses_df.to_csv("./.localdata/aua__jenza-cleaned-full.csv", index=False)